In [1]:
%cd /home/q123/Desktop/explo

import torch 
import logging
import logging.config

import yaml

from src.config import insertion_config,insert
from src.helpers import setup_experiment
from src.trainer import Trainer
from src.gibo.optim import GIBOptimizer
from src.optim import BOptimizer

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "Swimmer-v2"
kernel_name = "rbf"
n_init = 2
n_steps = 200


import gpytorch 

model_config = {
        "prior_mean":0,
        "ard_num_dims":18,
        "N_max":32,
        # "lengthscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.001),
        #"lengthscale_hyperprior":gpytorch.priors.torch_priors.UniformPrior(a=0.01,b=0.3),
        "lengthscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(loc=2.0,scale=1.0),
        "outputscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.001),
        "outputscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(loc=2.0,scale=1.0),
        "noise_constraint":None,
        "noise_hyperprior":None,
        
        }

In [3]:

model,objective_env = setup_experiment(env_name,model_config,n_init)
#optimizer = BOptimizer()
optimizer = GIBOptimizer(model)
#model.initialize(**hypers)
trainer = Trainer(model,objective_env,optimizer,n_steps)
trainer.run(report_freq=10)

/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Swimmer-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `Swimmer-v3`.
  logger.warn(


MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
TRUNCAAATING OLD DATAAAAAAAAAA
##############################
covar_lengthscale tensor([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],
       grad_fn=<SoftplusBackward>)                 covar_outputscale 2.0                 noise 0.009999999776482582
##############################
TRUNCAAATING OLD DATAAAAAAAAAA
##############################
covar_lengthscale tensor([[1.9997, 2.0151, 2.0176, 2.0135, 2.0514, 2.0192, 2.0177, 2.0232, 2.0090,
         0.4631, 2.0352, 2.0190, 2.0190, 2.0409, 2.0637, 1.9579, 2.0174, 2.0198]],
       grad_fn=<SoftplusBackward>)                 covar_outputscale 0.005731678102165461                 noise 0.009999999776482582
##############################
TRUNCAAATING OLD DATAAAAAAAAAA
##############################
covar_lengthscale tensor([[1.9393, 2.1404, 1.7687, 2.1149, 1.7566, 1.1593, 2.0191, 1.8973, 2.0340,
         1.7639, 2.1189, 2.1226, 2.1234, 2.1130, 0.4033, 2.0735, 

RuntimeError: Attempting to manually set a parameter value that is out of bounds of its current constraints, Positive(). Most likely, you want to do the following:
 likelihood = GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(better_lower_bound))

In [ ]:
config_file = "/home/q123/Desktop/explo/configs/swimmer.yaml"

with open(config_file, 'r') as f:
        cfg = yaml.load(f, Loader=yaml.Loader)

# Translate config dictionary.
cfg = insert(cfg, insertion_config)
model_config = cfg["optimizer_config"]["model_config"]
model_config

{'prior_mean': 0.0,
 'ard_num_dims': 'dim_search_space',
 'N_max': 32,
 'lengthscale_constraint': {'constraint': None, 'kwargs': None},
 'lengthscale_hyperprior': {'prior': gpytorch.priors.torch_priors.UniformPrior,
  'kwargs': {'a': 0.01, 'b': 0.3}},
 'outputscale_constraint': {'constraint': gpytorch.constraints.constraints.GreaterThan,
  'kwargs': {'lower_bound': 0.001}},
 'outputscale_hyperprior': {'prior': gpytorch.priors.torch_priors.NormalPrior,
  'kwargs': {'loc': 2.0, 'scale': 1.0}},
 'noise_constraint': {'constraint': None, 'kwargs': None},
 'noise_hyperprior': {'prior': None, 'kwargs': None}}